In [354]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder


# Preprocesamiento v2

Antes de entrenar el modelo, vamos a preparar el datatset.
### 1. **strip()** 
- Eliminar espacios en blanco en los nombres de las columnas

  **1.1** Transformación de última beca nombre**

  **1.2**Eliminar becas anteriores y mantener solo la última beca 

### 2. **Eliminar** 
         - Codigos silenciados innecesarias columnas para el analisis
         -  Silenciar columna ULT_BECA_PERIODO
### 3. **Preprocesamiento de Columnas**
 * **Nombre de centros**

         - Mantener el valor de la columna con mayor valor y los demás valores corregirlos al valor siguiente que es Loja.

#  5 **Conversión con LabelEncoder y OneHotEncoder** 

  * **Remplazar columnas con valores SI, NO a binarios**

Columnas : 
   - EST_CARR_SEGUNDA_REPROBACION
   - EST_CARR_TERCERA_REPROBACION
   - CUMPLE_SEGUNDA_LENGUA
  
  * **Formato de columna Edad a entero**

   - Convertir a valores Int
   - Pasar valores a binario

  * **Columna Rango Años de abandono de estudio**


    - Pasar valores a categóricos a promedio entre la suma de ellos *'promedioAñosAbandono'*

  * **Columna de Rango etario**

    - Pasar valores a categóricos a promedio entre la suma de ellos *'promedioRangoEtario'*

  * **Columna de nivel de carrera**

    - Pasar valores a categóricos a binarios 0 y 1

  * **Columna Modalidad**

    - Pasar valores a categóricos a bianrios 0, 1 y 2

  * **Columna nombre del centro**

    - Pasar valores a categóricos a binarios 0 y 1
    - 
  * **Columna nombre del Estado de la beca**

    - Pasar valores a categóricos a binarios 0 y 1
### - 4 **Revisión de valores nulos**

     

In [355]:
### primeras filas 
data = pd.read_csv("TAI_AN_ESTUDIANTE_12072024_v2.csv", low_memory=False)
# Mostrar las primeras filas del DataFrame
data.head(2)

,ID,ESTUDIANTE_FECHA_FALLECIMIENTO,ESTUDIANTE_CARRERA_NIVEL_COD,ESTUDIANTE_CARRERA_NIVEL,ESTUDIANTE_CARRERA_MODALID_COD,ESTUDIANTE_CARRERA_MODALIDAD,ESTUDIANTE_CARRERA_CODIGO,ESTUDIANTE_CARRERA_NOMBRE,ESTUDIANTE_CARRERA_MALLA_COD,ESTUDIANTE_CARRERA_PLAN_ESTUD,...,TOTAL_TAREAS_PLANIFICADAS,TOTAL_TAREAS_REALIZADAS,TOTAL_TAREAS_NO_REALIZADAS,TOTAL_TAREAS_CALIFICADAS,PROMEDIO_NOTA_TAREAS,TOTAL_REUNIONES,DIA_RECURRENTE_REUNIONES,TOTAL_REUNIONES_MINUTOS,TOTAL_PARTICIPACIONES,TOTAL_PARTICIPACIONES_MINUTOS
0,230,NaN,GR,GRADO,D,DISTANCIA,DERE_D1,DERECHO,DED3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,231,NaN,GR,GRADO,D,DISTANCIA,CONT_D1,CONTABILIDAD Y AUDITORIA,CAD1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [356]:
# Calcula el porcentaje total de valores nulos en el DataFrame dfv4
porcentaje_nulos_total = (data.isnull().sum().sum() / data.size) * 100

print(f"Porcentaje total de valores nulos en dfv4: {porcentaje_nulos_total:.2f}%")


Porcentaje total de valores nulos en dfv4: 55.09%


In [357]:
data.shape

(114519, 166)

# Valores nulos por columnas

In [358]:
# # Calcular el porcentaje de valores nulos por columna
# porcentajeNulosDataNum = data.isnull().sum()

# # Mostrar todo el resultado
# # print(porcentajeNulosDataNum.to_string())
# print(porcentajeNulosDataNum)


# 1 strip()

 
 
## 1.1 Eliminar espacios en blanco en los nombres de las columnas

Eliminar espacios en blanco al principio y al final de los nombres de las columnas

In [359]:
# Eliminar espacios en blanco al inicio y final de los nombres de las columnas
data.columns = data.columns.str.strip()

# Eliminar espacios en blanco al inicio y final de los valores en cada columna de tipo string
for column in data.select_dtypes(include='object').columns:
    data[column] = data[column].str.strip()


In [360]:
# data['ULT_BECA_NOMBRE'].unique()

### 1.2  Preparacio2n y limpieza de variable predictora ESTADO_BECA

### Limpieza de la variable predictora

In [361]:
# Reemplazar comillas simples internas por comillas dobles
data['ULT_BECA_NOMBRE'] = data['ULT_BECA_NOMBRE'].str.replace("'", '')

# Asegurar que el contenido de 'ACCESO A LAS TIC's' sea modificado correctamente
data['ULT_BECA_NOMBRE'] = data['ULT_BECA_NOMBRE'].str.replace(r"(?<=\S)'(?=\S)", '"', regex=True)
# data['ULT_BECA_NOMBRE'].unique()

In [362]:
# Función para extraer el nombre de la beca
def extract_beca_name(text):
    if isinstance(text, str) and 'BECA' in text:
        # Divide el texto por punto y coma y extrae las partes que contienen 'BECA'
        parts = [part.strip() for part in text.split(';') if 'BECA' in part]
        return '; '.join(parts)
    return 

# Aplicar la función a la columna 'column_name' para crear la nueva columna 'BECA_NOMBRE'
data['ESTADO_BECA'] = data['ULT_BECA_NOMBRE'].apply(extract_beca_name)

# Función para eliminar duplicados en cada fila
def eliminar_duplicados(be):
    if pd.isna(be):
        return be  # Devolver el valor tal cual si es NaN
    # Dividir la cadena en una lista
    be_list = be.split('; ')
    # Eliminar duplicados manteniendo el orden
    be_list_unicos = list(dict.fromkeys(be_list))
    # Volver a unir la lista en una cadena
    return '; '.join(be_list_unicos)
data['ESTADO_BECA'] = data['ESTADO_BECA'].fillna('Ninguno')

# Aplicar la función a la columna 'NOMBRE_BECA'
data['ESTADO_BECA'] = data['ESTADO_BECA'].apply(eliminar_duplicados)
# Eliminar la columna 'B'
data = data.drop(columns=['ULT_BECA_NOMBRE'],axis=1)

# print(data['ESTADO_BECA'].unique())


/var/folders/3n/wr348_g54y3dnpq24d5zdn9w0000gn/T/ipykernel_1952/1057161593.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['ESTADO_BECA'] = data['ULT_BECA_NOMBRE'].apply(extract_beca_name)


In [363]:
# Ver los valores únicos de la columna resultante
print(data['ESTADO_BECA'].unique())


['Ninguno' 'BECA RENDIMIENTO ACADEMICO' 'BECA UTPL'
 'BECA EXCELENCIA ACADEMICA' 'BECA FIDELIZACION' 'BECA NIVEL DE INGRESOS'
 'BECA GENERAL ECONOMICA' 'BECA DISCAPACIDAD' 'BECA CLUB UNIVERSITARIO'
 'BECA PUEBLOS NACIONALIDADES' 'BECA POLITICA DE CUOTAS'
 'BECA FAMILIAR PS' 'BECA CENTROS REHABILITACION' 'BECA DESEMPLEO'
 'BECA FAMILIAR HIJO DOCENTE' 'BECA RESPONS UNIVERSITARIA'
 'BECA FAMILIAR ESTUDIANTES' 'BECA NIVEL DE INGRESOS PS'
 'BECA MOVILIDAD ESTUDIANTIL' 'BECA FAMIL.HIJO ADMINISTRATIVO'
 'BECA ABANDERADOS'
 'BECA FAMILIAR ESTUDIANTES; BECA RENDIMIENTO ACADEMICO'
 'BECA DEPORTISTAS DESTACADOS' 'BECA CULTURAL' 'BECA DIOCESIS RELIGION'
 'BECA FUNCIONARIOS PS' 'BECA DESEMPLEO PT PS' 'BECA DIOCESIS DEL PAIS'
 'BECA NIVEL DE INGRESOS PT PS' 'BECA DESEMPLEO PS' 'BECA FAMILIAR PT PS'
 'BECA APOYO ECONOMICO GENERAL AREA' 'BECA REPRESENT ESTUDIANTIL'
 'BECA BACHILLERES DESTACADOS'
 'BECA FAMILIAR HIJO DOCENTE; BECA RENDIMIENTO ACADEMICO'
 'BECA TOMAS MORO' 'BECA DISCAPACIDAD PS'
 'BECA 

In [364]:
import pandas as pd

# Asegurarnos de que 'Ninguno' sea 'No' y el resto sea 'Sí'
data['ESTADO_BECA'] = data['ESTADO_BECA'].apply(lambda x: 'Si' if x == 'Ninguno' else 'No')

# Verifica los valores únicos de la columna
print(data['ESTADO_BECA'].unique())


['Si' 'No']


## 2.  Silenciar columnas innecesarias  

In [365]:
# Filtrar solo las columnas que contienen "_COD" en su nombre
columns_with_cod = data.filter(like='_CODIGO').columns.tolist() + \
             data.filter(like='CODIGO_').columns.tolist() + \
             data.filter(like='_COD').columns.tolist() 

# Calcular el porcentaje de valores nulos por columna
porcentajeNulosDataCat = (data.isnull().sum() / len(data)) * 100

# Filtrar las columnas con más del 50% de valores nulos
columnas_a_eliminar_nulos = porcentajeNulosDataCat[porcentajeNulosDataCat > 50].index.tolist()  # Asegúrate que sea una lista

# Lista de otras columnas a agregar a dfSilence
columnasNoConsideradas = ['EST_CARR_UNIDADES_REQUERIDAS','TOTAL_UNIDADES_ACUMULADAS',
                          'TOTAL_UNIDADES_PENDIENTES','CICLO_AVANCE_UNIDADES','CICLO_AVANCE_UNIDADES',
                         'TOTAL_ANIOS_EST_CAR','ULT_MAT_EST_CAR_PERIODO','ULT_MAT_EST_UTPL_PERIODO',
                         'ULT_BECA_PERIODO','TOTAL_ANIOS_EST_UTPL','ESTUDIANTE_CENTRO_ZONA','ESTUDIANTE_CENTRO_REGION',
                         'EST_CARR_MALLA_NOMBRE_CORTO','ESTUDIANTE_CARRERA_TIPO_MALLA','EST_CARR_PER_INGRESO','RANGO_SEMESTRE',
                         'RANGO_PORC_AVANCE','EST_CARR_MALLA_NOMBRE_LARGO','EST_CARR_TIPO_INGRESO','EST_CARR_ESTADO_PLAN_ESTUDIOS']  # Asegúrate de que si es vacía, sea una lista vacía válida

# Convertir la lista 'columnasNoConsideradas' en un DataFrame vacío para poder concatenarlo (opcional)
df_columnas_no_consideradas = pd.DataFrame(columns=columnasNoConsideradas)

# Crear un nuevo DataFrame dfSilence con las columnas seleccionadas
dfSilence = pd.concat([data[columns_with_cod], data[columnasNoConsideradas], data[columnas_a_eliminar_nulos]], axis=1)

# Eliminar las columnas del DataFrame original que han sido identificadas
data = data.drop(columns=columns_with_cod + columnas_a_eliminar_nulos + columnasNoConsideradas, errors='ignore')

# Crear un nuevo DataFrame data1 con las columnas restantes en data
data1 = data

# Verifica las columnas de data1 y de dfSilence
print("Columnas restantes en data1:")
# print(data1.columns)
print("FILAS Y COLUMNAS", data1.shape)


Columnas restantes en data1:
FILAS Y COLUMNAS (114519, 45)


In [366]:
data1.columns

Index(['ID', 'ESTUDIANTE_CARRERA_NIVEL', 'ESTUDIANTE_CARRERA_MODALIDAD',
       'ESTUDIANTE_CARRERA_NOMBRE', 'ESTUDIANTE_CARRERA_PLAN_ESTUD',
       'ESTUDIANTE_CARRERA_RESPONSABLE', 'ESTUDIANTE_CENTRO_NOMBRE',
       'EST_CARR_NUMERO_CICLOS', 'TOTAL_ASIGNATURAS_ACUMULADAS',
       'PORCENTAJE_AVANCE', 'ULTIMA_MATRICULA_PERIODO', 'CTAXCOBRAR_VIGENTE',
       'RETENCIONES_VIGENTES', 'EST_CARR_SEGUNDA_REPROBACION',
       'EST_CARR_TERCERA_REPROBACION', 'ESTUDIANTE_SANCIONES_VIGENTES',
       'CICLO_AVANCE_SEMESTRE', 'CICLO_ULTIMA_MATRICULA',
       'EST_CARR_INTENTOS_APROBACION', 'EST_CARR_INTENTOS_REPROBACION',
       'EST_CARR_ASIGNATURA_PENDIENTES', 'ULT_BECA_MONTO_REAL',
       'CARR_ACTUAL_GRADUADO', 'EST_CARR_ANIOS_ABANDONO', 'ESTUDIANTE_SEXO',
       'ESTUDIANTE_GAP', 'ESTUDIANTE_EDAD', 'CURSA_CARRERA_GRADO',
       'CURSA_CARRERA_POSGR', 'CURSA_CARRERA_TEC', 'TOTAL_ULT_BECAS_ASIGN',
       'TITULADO_CARRERA_GR', 'TITULADO_PROG_PS', 'EST_CARR_CANT_ASG_3ERA_REP',
       'EST_CARR_

#  Data catego2ica limpieza

In [367]:
# Contar columnas categóricas (objetos y categorías)
categorical_columns = data1.select_dtypes(include=['object']).columns
dataCateg = data1[categorical_columns].copy() 

# Imprimir resultados
dataCateg.head()


,ESTUDIANTE_CARRERA_NIVEL,ESTUDIANTE_CARRERA_MODALIDAD,ESTUDIANTE_CARRERA_NOMBRE,ESTUDIANTE_CARRERA_RESPONSABLE,ESTUDIANTE_CENTRO_NOMBRE,ULTIMA_MATRICULA_PERIODO,CTAXCOBRAR_VIGENTE,RETENCIONES_VIGENTES,EST_CARR_SEGUNDA_REPROBACION,EST_CARR_TERCERA_REPROBACION,...,TITULADO_PROG_PS,ESTUDIANTE_FALLECIDO,RANGO_ANIOS_ABANDONO,RANGO_ETARIO,IMPEDIMENTOS,DESCUENTO_REINGRESO,TIENE_AUTORIZACION_TM,CUMPLE_SEGUNDA_LENGUA,EST_CARR_OTRA_MALLA_SIN_MATRIC,ESTADO_BECA
0,GRADO,DISTANCIA,DERECHO,FACULTAD DE CIENCIAS JURÍDICAS Y POLÍTICAS,COCA,OCT/2015 - FEB/2016,NO,NO,NO,NO,...,NO,NO,6 A 10,51 A 65,NO,NO,No,No,NO,Si
1,GRADO,DISTANCIA,CONTABILIDAD Y AUDITORIA,FACULTAD DE CIENCIAS ECONÓMICAS Y EMPRESARIALES,MACARA,ABR/2008 - AGO/2008,NO,NO,NO,NO,...,NO,NO,16 A 20,31 A 40,NO,NO,No,No,NO,Si
2,GRADO,DISTANCIA,DERECHO,FACULTAD DE CIENCIAS JURÍDICAS Y POLÍTICAS,QUITO-TUMBACO,ABR/2011 - AGO/2011,NO,NO,NO,NO,...,NO,NO,11 A 15,31 A 40,NO,NO,No,No,NO,Si
3,GRADO,DISTANCIA,COMUNICACION,"FACULTAD DE CIENCIAS SOCIALES, EDUCACIÓN Y HUM...",PORTOVIEJO,OCT/2013 - FEB/2014,NO,NO,NO,NO,...,NO,NO,11 A 15,41 A 50,NO,NO,No,No,NO,Si
4,GRADO,DISTANCIA,CONTABILIDAD Y AUDITORIA,FACULTAD DE CIENCIAS ECONÓMICAS Y EMPRESARIALES,CUENCA,ABR/2014 - AGO/2014,NO,NO,NO,SI,...,NO,NO,6 A 10,51 A 65,SI,NO,No,No,NO,Si


### 1. Transformación columna centros 

In [368]:
# dataCateg['ESTUDIANTE_CENTRO_NOMBRE'].value_counts()

In [369]:
# Cambiar valores a 'QUITO' y 'LOJA'
dataCateg['ESTUDIANTE_CENTRO_NOMBRE'] = dataCateg['ESTUDIANTE_CENTRO_NOMBRE'].apply(lambda x: x if x == 'QUITO' else 'LOJA')

# Verificar los resultados
print(dataCateg['ESTUDIANTE_CENTRO_NOMBRE'].value_counts())


ESTUDIANTE_CENTRO_NOMBRE
LOJA     101893
QUITO     12626
Name: count, dtype: int64


# DATA Numérica Limpieza

In [370]:
# Contar columnas numéricas
dataNumerica = data1.select_dtypes(include=['number']).columns
dataNum = data1[dataNumerica].copy()

# Imprimir resultados
dataNum.head(2)


,ID,ESTUDIANTE_CARRERA_PLAN_ESTUD,EST_CARR_NUMERO_CICLOS,TOTAL_ASIGNATURAS_ACUMULADAS,PORCENTAJE_AVANCE,CICLO_AVANCE_SEMESTRE,CICLO_ULTIMA_MATRICULA,EST_CARR_INTENTOS_APROBACION,EST_CARR_INTENTOS_REPROBACION,EST_CARR_ASIGNATURA_PENDIENTES,ULT_BECA_MONTO_REAL,EST_CARR_ANIOS_ABANDONO,ESTUDIANTE_EDAD,TOTAL_ULT_BECAS_ASIGN,EST_CARR_CANT_ASG_3ERA_REP,EST_CARR_CANT_ASG_2DA_REP,TOTAL_ANIOS
0,230,1,10,1,0.0,0.0,1.0,1.0,5.0,54.0,80.0,8,54.0,1.0,0,0,9
1,231,1,10,0,0.0,0.0,1.0,NaN,5.0,57.0,NaN,15,35.0,NaN,0,0,16


## 3.5 Formato de columna Edad a entero

In [371]:
dataEdad = dataNum[dataNum['ESTUDIANTE_EDAD'].isnull()]
# dataEdad
media_edad = dataNum['ESTUDIANTE_EDAD'].mean()

# Rellena los NaN con la media y convertir a enteros
dataNum['ESTUDIANTE_EDAD'] = dataNum['ESTUDIANTE_EDAD'].fillna(media_edad).astype(int)



# 4. REVISAR VALORES NULOS

## Data categórica nulos

In [372]:
pd.set_option('display.max_rows', None)

valores_nulos_por_columna = dataCateg.isnull().sum()
# Imprimir todas las columnas con su respectiva cantidad de valores nulos
print("Todas las columnas y su cantidad de valores nulos:")
print(valores_nulos_por_columna)


Todas las columnas y su cantidad de valores nulos:
ESTUDIANTE_CARRERA_NIVEL             0
ESTUDIANTE_CARRERA_MODALIDAD         0
ESTUDIANTE_CARRERA_NOMBRE            0
ESTUDIANTE_CARRERA_RESPONSABLE       0
ESTUDIANTE_CENTRO_NOMBRE             0
ULTIMA_MATRICULA_PERIODO          8136
CTAXCOBRAR_VIGENTE                   0
RETENCIONES_VIGENTES                 0
EST_CARR_SEGUNDA_REPROBACION         0
EST_CARR_TERCERA_REPROBACION         0
ESTUDIANTE_SANCIONES_VIGENTES        0
CARR_ACTUAL_GRADUADO                 0
ESTUDIANTE_SEXO                      0
ESTUDIANTE_GAP                       0
CURSA_CARRERA_GRADO                  0
CURSA_CARRERA_POSGR                  0
CURSA_CARRERA_TEC                    0
TITULADO_CARRERA_GR                  0
TITULADO_PROG_PS                     0
ESTUDIANTE_FALLECIDO                 0
RANGO_ANIOS_ABANDONO                 0
RANGO_ETARIO                         0
IMPEDIMENTOS                         0
DESCUENTO_REINGRESO                  0
TIENE_AUTORIZ

### DATA numéria nulos 

In [373]:
# pd.set_option('display.max_rows',  None)

valores_nulos_por_columna = dataNum.isnull().sum()
# Imprimir todas las columnas con su respectiva cantidad de valores nulos
print("Todas las columnas y su cantidad de valores nulos:")
print(valores_nulos_por_columna)


Todas las columnas y su cantidad de valores nulos:
ID                                    0
ESTUDIANTE_CARRERA_PLAN_ESTUD         0
EST_CARR_NUMERO_CICLOS                0
TOTAL_ASIGNATURAS_ACUMULADAS          0
PORCENTAJE_AVANCE                     1
CICLO_AVANCE_SEMESTRE               152
CICLO_ULTIMA_MATRICULA             1748
EST_CARR_INTENTOS_APROBACION      47869
EST_CARR_INTENTOS_REPROBACION     28572
EST_CARR_ASIGNATURA_PENDIENTES      514
ULT_BECA_MONTO_REAL               49291
EST_CARR_ANIOS_ABANDONO               0
ESTUDIANTE_EDAD                       0
TOTAL_ULT_BECAS_ASIGN             49291
EST_CARR_CANT_ASG_3ERA_REP            0
EST_CARR_CANT_ASG_2DA_REP             0
TOTAL_ANIOS                           0
dtype: int64


In [374]:
print(dataNum.shape)
print(dataCateg.shape)

(114519, 17)
(114519, 28)


# Limpieza de columnas

### Relleno de de valores de columnas con nulos

In [375]:

dataNum['CICLO_AVANCE_SEMESTRE'] = dataNum['CICLO_AVANCE_SEMESTRE'].fillna(0)

dataNum['ULT_BECA_MONTO_REAL'] = dataNum['ULT_BECA_MONTO_REAL'].fillna(0)

dataNum['PORCENTAJE_AVANCE'] = dataNum['PORCENTAJE_AVANCE'].fillna(0)

dataNum['CICLO_ULTIMA_MATRICULA'] = dataNum['CICLO_ULTIMA_MATRICULA'].fillna(0)

dataNum['TOTAL_ULT_BECAS_ASIGN'] = dataNum['TOTAL_ULT_BECAS_ASIGN'].fillna(0)

dataNum['EST_CARR_ASIGNATURA_PENDIENTES'] = dataNum['EST_CARR_ASIGNATURA_PENDIENTES'].fillna(0)

dataNum['EST_CARR_INTENTOS_APROBACION'] = dataNum['EST_CARR_INTENTOS_APROBACION'].fillna(0)

dataNum['EST_CARR_INTENTOS_REPROBACION'] = dataNum['EST_CARR_INTENTOS_REPROBACION'].fillna(0)


In [376]:
dataCateg.head()

,ESTUDIANTE_CARRERA_NIVEL,ESTUDIANTE_CARRERA_MODALIDAD,ESTUDIANTE_CARRERA_NOMBRE,ESTUDIANTE_CARRERA_RESPONSABLE,ESTUDIANTE_CENTRO_NOMBRE,ULTIMA_MATRICULA_PERIODO,CTAXCOBRAR_VIGENTE,RETENCIONES_VIGENTES,EST_CARR_SEGUNDA_REPROBACION,EST_CARR_TERCERA_REPROBACION,...,TITULADO_PROG_PS,ESTUDIANTE_FALLECIDO,RANGO_ANIOS_ABANDONO,RANGO_ETARIO,IMPEDIMENTOS,DESCUENTO_REINGRESO,TIENE_AUTORIZACION_TM,CUMPLE_SEGUNDA_LENGUA,EST_CARR_OTRA_MALLA_SIN_MATRIC,ESTADO_BECA
0,GRADO,DISTANCIA,DERECHO,FACULTAD DE CIENCIAS JURÍDICAS Y POLÍTICAS,LOJA,OCT/2015 - FEB/2016,NO,NO,NO,NO,...,NO,NO,6 A 10,51 A 65,NO,NO,No,No,NO,Si
1,GRADO,DISTANCIA,CONTABILIDAD Y AUDITORIA,FACULTAD DE CIENCIAS ECONÓMICAS Y EMPRESARIALES,LOJA,ABR/2008 - AGO/2008,NO,NO,NO,NO,...,NO,NO,16 A 20,31 A 40,NO,NO,No,No,NO,Si
2,GRADO,DISTANCIA,DERECHO,FACULTAD DE CIENCIAS JURÍDICAS Y POLÍTICAS,LOJA,ABR/2011 - AGO/2011,NO,NO,NO,NO,...,NO,NO,11 A 15,31 A 40,NO,NO,No,No,NO,Si
3,GRADO,DISTANCIA,COMUNICACION,"FACULTAD DE CIENCIAS SOCIALES, EDUCACIÓN Y HUM...",LOJA,OCT/2013 - FEB/2014,NO,NO,NO,NO,...,NO,NO,11 A 15,41 A 50,NO,NO,No,No,NO,Si
4,GRADO,DISTANCIA,CONTABILIDAD Y AUDITORIA,FACULTAD DE CIENCIAS ECONÓMICAS Y EMPRESARIALES,LOJA,ABR/2014 - AGO/2014,NO,NO,NO,SI,...,NO,NO,6 A 10,51 A 65,SI,NO,No,No,NO,Si


In [377]:
dataCateg['ULTIMA_MATRICULA_PERIODO'] = dataCateg['ULTIMA_MATRICULA_PERIODO'].fillna('0000')


In [378]:

# # Filtrar las filas donde la columna 'ULTIMA_MATRICULA_PERIODO' tiene valores nulos
# filterNul = dataNum[dataNum['TOTAL_ANIOS_EST_CAR'].isna()]

# # Contar el número de filas que contienen valores nulos en esa columna
# # print(filterNul.shape[0])
# filterNul.head()


# GUARDAR Nuevo dataframe V2 limpio Con valores catego2ricos

In [379]:
# Unir los DataFrames
dataUnido = pd.concat([dataNum, dataCateg], axis=1)

# Guardar el DataFrame resultante en un archivo CSV
dataUnido.to_csv('dataCleanV2.csv', index=False)


In [380]:
# dataUnido.isna().sum()

In [381]:
dataUnido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114519 entries, 0 to 114518
Data columns (total 45 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              114519 non-null  int64  
 1   ESTUDIANTE_CARRERA_PLAN_ESTUD   114519 non-null  int64  
 2   EST_CARR_NUMERO_CICLOS          114519 non-null  int64  
 3   TOTAL_ASIGNATURAS_ACUMULADAS    114519 non-null  int64  
 4   PORCENTAJE_AVANCE               114519 non-null  float64
 5   CICLO_AVANCE_SEMESTRE           114519 non-null  float64
 6   CICLO_ULTIMA_MATRICULA          114519 non-null  float64
 7   EST_CARR_INTENTOS_APROBACION    114519 non-null  float64
 8   EST_CARR_INTENTOS_REPROBACION   114519 non-null  float64
 9   EST_CARR_ASIGNATURA_PENDIENTES  114519 non-null  float64
 10  ULT_BECA_MONTO_REAL             114519 non-null  float64
 11  EST_CARR_ANIOS_ABANDONO         114519 non-null  int64  
 12  ESTUDIANTE_EDAD 

#  **Conversión con LabelEncoder y OneHotEncoder** 

  * **Remplazar columnas con valores SI, NO a binarios**

    Columnas : 
       - EST_CARR_SEGUNDA_REPROBACION
       - EST_CARR_TERCERA_REPROBACION
       - CUMPLE_SEGUNDA_LENGUA
  
  * **Formato de columna Edad a entero**

   - Convertir a valores Int
   - Pasar valores a binario

  * **Columna Rango Años de abandono de estudio**


    - Pasar valores a categóricos a promedio entre la suma de ellos *'promedioAñosAbandono'*

  * **Columna de Rango etario**

    - Pasar valores a categóricos a promedio entre la suma de ellos *'promedioRangoEtario'*

  * **Columna de nivel de carrera**

    - Pasar valores a categóricos a binarios 0 y 1

  * **Columna Modalidad**

    - Pasar valores a categóricos a bianrios 0, 1 y 2

  * **Columna nombre del centro**

    - Pasar valores a categóricos a binarios 0 y 1
    - 
  * **Columna nombre del Estado de la beca**

    - Pasar valores a categóricos a binarios 0 y 1
### **Revisión de valores nulos**

   * Columnas para conversión de categórica a numérica CON DUMMIES

     * ESTUDIANTE_CARRERA_NIVEL
     * ESTUDIANTE_CARRERA_MODALIDAD
     * ESTUDIANTE_CARRERA_RESPONSABLE
     * ESTUDIANTE_CENTRO_NOMBRE




### 1. ESTUDIANTE_CARRERA_NIVEL

In [382]:
# O convertir a binario
dataUnido['ESTUDIANTE_CARRERA_NIVEL'] = dataUnido['ESTUDIANTE_CARRERA_NIVEL'].map({'GRADO': 0, 'POSGRADO': 1})


### 2. ESTUDIANTE_CARRERA_MODALIDAD


In [383]:
# Convertir a tipo categórico
dataUnido['ESTUDIANTE_CARRERA_MODALIDAD'] = dataUnido['ESTUDIANTE_CARRERA_MODALIDAD'].map({'DISTANCIA': 0, 'PRESENCIAL': 1,'EN LINEA': 2})


### 3. RANGO_ANIOS_ABANDONO

# Corrección de variables categóricas a  DUMMIES

In [384]:

# Asegúrate de que todos los valores sean cadenas de texto
dataUnido['RANGO_ANIOS_ABANDONO'] = dataUnido['RANGO_ANIOS_ABANDONO'].astype(str)

# Limpia los espacios alrededor del guion
dataUnido['RANGO_ANIOS_ABANDONO'] = dataUnido['RANGO_ANIOS_ABANDONO'].str.replace(r'\s*-\s*', ' A ', regex=True)

# Diccionario de rangos a valores promedio
ingresos_map = {
    '6 A 10':8, 
    '16 A 20':18,  
    '11 A 15':13, 
    '0':0, 
    '1 A 5':3, 
    '21 A 25':23, 
    '26 A 30':33
}

# Reemplaza los valores en la columna
dataUnido['promedioAñosAbandono'] = dataUnido['RANGO_ANIOS_ABANDONO'].map(ingresos_map)

# Convierte la nueva columna a float
dataUnido['promedioAñosAbandono'] = dataUnido['promedioAñosAbandono'].astype(int)

# Verifica si hay algún NaN resultante y muestra los casos
print(dataUnido['promedioAñosAbandono'].unique())


[ 8 18 13  0  3 23 33]


In [385]:
# Mapeo de rangos etarios a categorías
rango_etario_map = {
    '51 A 65': 'Adulto',
    '31 A 40': 'Adulto',
    '41 A 50': 'Adulto',
    '25 A 30': 'Joven',
    'Mayores a 65': 'Adulto mayor',
    '16 A 24': 'Joven',
    'Fuera de rango': 'Otro'
}

# Aplicar el mapeo
dataUnido['rango_etario_categoria'] = dataUnido['RANGO_ETARIO'].map(rango_etario_map)

# Verificar los resultados
print(dataUnido['rango_etario_categoria'].unique())


# Aplicar pd.get_dummies para convertir la columna 'RANGO_ETARIO' en columnas binarias
dataUnido = pd.get_dummies(dataUnido, columns=['rango_etario_categoria'], prefix='rango_Etario', drop_first=False)

# Verificar las columnas generadas
# print(dataUnido.columns)


['Adulto' 'Joven' 'Adulto mayor' 'Otro']


# Conversión columna Sexo

In [386]:
# Inicializar LabelEncoder y transformar la columna 'ESTUDIANTE_SEXO'
le = LabelEncoder()
dataUnido['ESTUDIANTE_SEXO'] = le.fit_transform(dataUnido['ESTUDIANTE_SEXO'])

# Contar los valores únicos en 'ESTUDIANTE_SEXO' y almacenarlos en 'conteo_sexo'
conteoSexo = data['ESTUDIANTE_SEXO'].value_counts()
conteo_sexo = dataUnido['ESTUDIANTE_SEXO'].value_counts()

# Mostrar el resultado
print(conteoSexo)
print(conteo_sexo)


ESTUDIANTE_SEXO
MUJER            62314
HOMBRE           49753
NO DISPONIBLE     2452
Name: count, dtype: int64
ESTUDIANTE_SEXO
1    62314
0    49753
2     2452
Name: count, dtype: int64


### Conversión centro nombre a binarios

In [387]:

# Mapear 'QUITO' a 0 y 'LOJA' a 1
dataUnido['ESTUDIANTE_CENTRO_NOMBRE'] = dataUnido['ESTUDIANTE_CENTRO_NOMBRE'].map({'QUITO': 0, 'LOJA': 1})
dataUnido['ESTUDIANTE_CENTRO_NOMBRE'] = dataUnido['ESTUDIANTE_CENTRO_NOMBRE'].astype(int)
dataUnido['ESTUDIANTE_CENTRO_NOMBRE'].value_counts()


ESTUDIANTE_CENTRO_NOMBRE
1    101893
0     12626
Name: count, dtype: int64

### Conversión e variables predictiva a valores binarios 0 y 1


In [388]:

# # Convertir los valores a 1 y 0
# dataUnido['ESTADO_BECA'] = dataUnido['ESTADO_BECA'].apply(lambda x: 0 if x == 'Ninguno' else 1)
# # Verificar los cambios

# print(dataUnido['ESTADO_BECA'].value_counts())

## 2 Conversión de valores SI , NO en binarios


In [389]:
dataUnido = dataUnido.replace({'SI': 1, 'NO': 0, 'Si': 1, 'No': 0})
dataUnido = dataUnido.infer_objects(copy=False)

/var/folders/3n/wr348_g54y3dnpq24d5zdn9w0000gn/T/ipykernel_1952/2430805621.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataUnido = dataUnido.replace({'SI': 1, 'NO': 0, 'Si': 1, 'No': 0})


In [390]:
dataUnido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114519 entries, 0 to 114518
Data columns (total 50 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              114519 non-null  int64  
 1   ESTUDIANTE_CARRERA_PLAN_ESTUD   114519 non-null  int64  
 2   EST_CARR_NUMERO_CICLOS          114519 non-null  int64  
 3   TOTAL_ASIGNATURAS_ACUMULADAS    114519 non-null  int64  
 4   PORCENTAJE_AVANCE               114519 non-null  float64
 5   CICLO_AVANCE_SEMESTRE           114519 non-null  float64
 6   CICLO_ULTIMA_MATRICULA          114519 non-null  float64
 7   EST_CARR_INTENTOS_APROBACION    114519 non-null  float64
 8   EST_CARR_INTENTOS_REPROBACION   114519 non-null  float64
 9   EST_CARR_ASIGNATURA_PENDIENTES  114519 non-null  float64
 10  ULT_BECA_MONTO_REAL             114519 non-null  float64
 11  EST_CARR_ANIOS_ABANDONO         114519 non-null  int64  
 12  ESTUDIANTE_EDAD 

## Tratamiento de la columna de carreras 

In [391]:
# Contar las frecuencias de cada carrera
frecuencias = dataUnido['ESTUDIANTE_CARRERA_NOMBRE'].value_counts()

# Seleccionar un umbral para las carreras más frecuentes (e.g., las 3 más comunes)
umbral = 3
carreras_frecuentes = frecuencias.nlargest(umbral).index
# print("Carreras s:", dataUnido['ESTUDIANTE_CARRERA_NOMBRE'].value_counts())
print("Carreras más frecuentes:", carreras_frecuentes)


Carreras más frecuentes: Index(['DERECHO', 'CONTABILIDAD Y AUDITORIA', 'EDUCACION BASICA'], dtype='object', name='ESTUDIANTE_CARRERA_NOMBRE')


### Agrupar las menos frecuentes como "OTRAS":

In [392]:
# Reemplazar las carreras menos frecuentes con "OTRAS"
dataUnido['ESTUDIANTE_CARRERA_NOMBRE'] = dataUnido['ESTUDIANTE_CARRERA_NOMBRE'].apply(
    lambda x: x if x in carreras_frecuentes else 'OTRAS'
)

# Verificar las categorías resultantes
print(dataUnido['ESTUDIANTE_CARRERA_NOMBRE'].unique())


['DERECHO' 'CONTABILIDAD Y AUDITORIA' 'OTRAS' 'EDUCACION BASICA']


In [393]:
print(dataUnido.columns)

Index(['ID', 'ESTUDIANTE_CARRERA_PLAN_ESTUD', 'EST_CARR_NUMERO_CICLOS',
       'TOTAL_ASIGNATURAS_ACUMULADAS', 'PORCENTAJE_AVANCE',
       'CICLO_AVANCE_SEMESTRE', 'CICLO_ULTIMA_MATRICULA',
       'EST_CARR_INTENTOS_APROBACION', 'EST_CARR_INTENTOS_REPROBACION',
       'EST_CARR_ASIGNATURA_PENDIENTES', 'ULT_BECA_MONTO_REAL',
       'EST_CARR_ANIOS_ABANDONO', 'ESTUDIANTE_EDAD', 'TOTAL_ULT_BECAS_ASIGN',
       'EST_CARR_CANT_ASG_3ERA_REP', 'EST_CARR_CANT_ASG_2DA_REP',
       'TOTAL_ANIOS', 'ESTUDIANTE_CARRERA_NIVEL',
       'ESTUDIANTE_CARRERA_MODALIDAD', 'ESTUDIANTE_CARRERA_NOMBRE',
       'ESTUDIANTE_CARRERA_RESPONSABLE', 'ESTUDIANTE_CENTRO_NOMBRE',
       'ULTIMA_MATRICULA_PERIODO', 'CTAXCOBRAR_VIGENTE',
       'RETENCIONES_VIGENTES', 'EST_CARR_SEGUNDA_REPROBACION',
       'EST_CARR_TERCERA_REPROBACION', 'ESTUDIANTE_SANCIONES_VIGENTES',
       'CARR_ACTUAL_GRADUADO', 'ESTUDIANTE_SEXO', 'ESTUDIANTE_GAP',
       'CURSA_CARRERA_GRADO', 'CURSA_CARRERA_POSGR', 'CURSA_CARRERA_TEC',
       'T

#### Aplicar One-Hot Encoding a la columna agrupada: de carreras


In [394]:
# Crear variables dummies para las carreras
dataUnido = pd.get_dummies(dataUnido, columns=['ESTUDIANTE_CARRERA_NOMBRE'], prefix='CARRERA')


#### Aplicar One-Hot Encoding a la columna agrupada: de facultades


In [395]:
# Aplicar One-Hot Encoding a las facultades
dataUnido = pd.get_dummies(dataUnido, columns=['ESTUDIANTE_CARRERA_RESPONSABLE'], prefix='FACULTAD')

### Tratamiento de periodo ultima matricula del periodo

In [396]:
# Extraer los años de inicio y fin como listas
dataUnido['ultimaMatAño'] = dataUnido['ULTIMA_MATRICULA_PERIODO'].str.extract(r'(\d{4})', expand=False)
# print(dataUnido['AÑO_INICIO'].unique())
# print(dataUnido[['ID', 'AÑO_INICIO']].head(10))


dataUnido['ultimaMatAño'] = dataUnido['ultimaMatAño'].fillna(0)
print(dataUnido['ultimaMatAño'].unique())

dataUnido['ultimaMatAño'] = dataUnido['ultimaMatAño'].astype(int)

# print(dataUnido['AÑO'].head())


['2015' '2008' '2011' '2013' '2014' '2012' '2024' '2018' '2005' '0000'
 '2016' '2019' '2022' '2006' '2010' '2009' '2007' '2021' '2023' '2017'
 '2020' '2004' '2003' '2000' '2002' '1998' '2001' '1999' '1996' '1994'
 '1997']


In [397]:
# Mapeo de rangos etarios a categorías
rango_etario_map = {
    0 : 'Ninguno',
'CENTRO EXTRANJERO':'EXTRANJERO', 
'PERSONAS CON DISCAPACIDAD':'PERSONAS CON DISCAPACIDAD',
'PPL':'PPL'
}

# Aplicar el mapeo
dataUnido['est_GAP'] = dataUnido['ESTUDIANTE_GAP'].map(rango_etario_map)

# Verificar los resultados
print(dataUnido['est_GAP'].unique())

dataUnido = pd.get_dummies(dataUnido, columns=['est_GAP'], prefix='est_GAP', drop_first=False)


['Ninguno' 'EXTRANJERO' 'PERSONAS CON DISCAPACIDAD' 'PPL']


In [398]:
# Eliminar las columnas especificadas
dataUnido = dataUnido.drop(columns=['RANGO_ANIOS_ABANDONO', 'RANGO_ETARIO','ESTUDIANTE_GAP','ULTIMA_MATRICULA_PERIODO'])


# Guarad CSV transformado a numérico CON VARIABLES DUMIIES

In [399]:
dataUnido = dataUnido.astype({col: 'int' for col in dataUnido.select_dtypes(include=['bool']).columns})

# Guardar el DataFrame resultante en un archivo CSV
dataUnido.to_csv('dataDummiesV2.csv', index=False, sep=";")


In [400]:
dataUnido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114519 entries, 0 to 114518
Data columns (total 59 columns):
 #   Column                                                           Non-Null Count   Dtype  
---  ------                                                           --------------   -----  
 0   ID                                                               114519 non-null  int64  
 1   ESTUDIANTE_CARRERA_PLAN_ESTUD                                    114519 non-null  int64  
 2   EST_CARR_NUMERO_CICLOS                                           114519 non-null  int64  
 3   TOTAL_ASIGNATURAS_ACUMULADAS                                     114519 non-null  int64  
 4   PORCENTAJE_AVANCE                                                114519 non-null  float64
 5   CICLO_AVANCE_SEMESTRE                                            114519 non-null  float64
 6   CICLO_ULTIMA_MATRICULA                                           114519 non-null  float64
 7   EST_CARR_INTENTOS_APROBACION 

In [401]:
dataUnido['ESTADO_BECA'].unique()

array([1, 0])

# CAMPOS CATEGÓRICOS Y NUMÉRICOS

| Columna                                  | Descripción                                                         | Transformación                  | Formato      | Acción Realizada                 | 
|------------------------------------------|---------------------------------------------------------------------|---------------------------------|--------------|-----------------------------------|
| ESTUDIANTE_CARRERA_NIVEL                | Nivel de carrera del estudiante | OneHotEncoder                   | Binario (0/1)  | Columna dividida en categorías en más columnas   | 
| ESTUDIANTE_CARRERA_MODALIDAD            | Modalidad de estudio | OneHotEncoder                   | Binario (0/1) | Columna dividida en categorías  en más columnas | 
| ESTUDIANTE_CARRERA_RESPONSABLE          | Facultad responsable de la carrera | OneHotEncoder            | Binario (0/1) | Columna dividida en categorías  en más columnas  | 
| ESTUDIANTE_CENTRO_NOMBRE                | Nombre del centro de estudios | OneHotEncoder                   | Binario (0/1) | Columna dividida en categorías en más columnas |
| PROMEDIO_GENERAL                         | Promedio académico del estudiante | Procesado de los valores nulos por 0   | Decimal      | No se transformó, se relleno campos vacios |
| EST_CARR_SEGUNDA_REPROBACION            | Número de materias reprobadas por segunda vez | Binario   | Entero       | Numerización de valores SI, NO pora binarios | 
| EST_CARR_TERCERA_REPROBACION            | Número de materias reprobadas por tercera vez  | Binario | Entero       | Numerización de valores SI, NO pora binarios | 
| ESTUDIANTE_SEXO                         | Sexo del estudiante | OneHotEncoder                   | Binario (0/1) | Numerización de valores a binarios   | 
| ESTUDIANTE_EDAD                         | Edad del estudiante                                                | Transformación de formato | Entero       | Se cambio el formato float a Int | 
| CUMPLE_SEGUNDA_LENGUA                   | Indica si cumple con un segundo idioma                             | Binario  | Binario (0/1) | Numerización de valores SI, NO pora binarios    |
| ESTADO_BECA                             | Estado de beca del estudiante| Valores  con 1 y los vacio con 0 | Binario (0/1) | Relleno con valores 0 cuando esta con nan | 
